<h1> <b> Lab - 6 </b> </h1>
<h3>Logistic Regression on MNIST dataset using Tensorflow.</h3>
<br>

<br/><h2> <b> Import Necessary libraries.</b> </h2><br/>

In [4]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
import numpy as np

<br/><h2> <b> Initializing the hyperparameters and data set parameters. </b> </h2><br/>

In [5]:
# MNIST dataset parameters.

# 0 to 9 digits
num_classes = 10 

# 28*28
num_features = 784

# Training parameters.
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

<br/><h2> <b> Loading and preparing the MNIST dataset.</b> </h2><br/>

In [9]:
from tensorflow.keras.datasets import mnist

(inputs_train, targets_train), (inputs_test, targets_test) = mnist.load_data()

# Convert to float32.
inputs_train, inputs_test = np.array(inputs_train, np.float32), np.array(inputs_test, np.float32)

# Flatten images to 1-D vector of 784 features (28*28).
inputs_train, inputs_test = inputs_train.reshape([-1, num_features]), inputs_test.reshape([-1, num_features])

# Normalize images value from [0, 255] to [0, 1].
inputs_train, inputs_test = inputs_train / 255., inputs_test / 255.

<br/><h2> <b> Shuffling and batching the data.</b> </h2><br/>

In [11]:
# Use tf.data API to shuffle and batch data.

train_data = tf.data.Dataset.from_tensor_slices((inputs_train, targets_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

<br/><h2> <b> Initializing the weights and biases.</b> </h2><br/>

In [12]:
# Weight of shape [784, 10], the 28*28 image features, and a total number of classes.
W = tf.Variable(tf.ones([num_features, num_classes]), name = "weight")

# Bias of shape [10], the total number of classes.

b = tf.Variable(tf.zeros([num_classes]), name = "bias")

<br/><h2> <b> Defining the logistic regression and cost function.</b> </h2><br/>

In [13]:
# Logistic regression (Wx + b).

def logistic_regression(x):

    # Apply softmax to normalize the logits to a probability distribution.

    return tf.nn.softmax(tf.matmul(x, W) + b)

# Cross-Entropy loss function.

def cross_entropy(y_pred, y_true):

    # Encode label to a one hot vector.

    y_true = tf.one_hot(y_true, depth=num_classes)

    # Clip prediction values to avoid log(0) error.

    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    # Compute cross-entropy.

    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

<br/><h2> <b> Defining optimizers and accuracy metrics.</b> </h2><br/>

In [14]:
# Accuracy metric.

def accuracy(y_pred, y_true):

  # Predicted class is the index of the highest score in prediction vector (i.e. argmax).

  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))

  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Stochastic gradient descent optimizer.

optimizer = tf.optimizers.SGD(learning_rate)

<br/><h2> <b> Optimization process and updating weights and biases.</b> </h2><br/>

In [15]:
# Optimization process. 

def run_optimization(x, y):

# Wrap computation inside a GradientTape for automatic differentiation.

    with tf.GradientTape() as g:

        pred = logistic_regression(x)

        loss = cross_entropy(pred, y)

    # Compute gradients.

    gradients = g.gradient(loss, [W, b])

    # Update W and b following gradients.

    optimizer.apply_gradients(zip(gradients, [W, b]))

<br/><h2> <b> The traning loop.</b> </h2><br/>

In [16]:
# Run training for the given number of steps.

for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):

    # Run the optimization to update W and b values.

    run_optimization(batch_x, batch_y)

    if step % display_step == 0:

        pred = logistic_regression(batch_x)

        loss = cross_entropy(pred, batch_y)

        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 528.999451, accuracy: 0.839844
step: 100, loss: 108.224426, accuracy: 0.894531
step: 150, loss: 305.229431, accuracy: 0.730469
step: 200, loss: 135.933777, accuracy: 0.890625
step: 250, loss: 80.160927, accuracy: 0.921875
step: 300, loss: 324.033691, accuracy: 0.746094
step: 350, loss: 71.844398, accuracy: 0.917969
step: 400, loss: 192.744522, accuracy: 0.820312
step: 450, loss: 58.223103, accuracy: 0.921875
step: 500, loss: 81.265228, accuracy: 0.910156
step: 550, loss: 100.616714, accuracy: 0.851562
step: 600, loss: 142.453857, accuracy: 0.851562
step: 650, loss: 72.150513, accuracy: 0.906250
step: 700, loss: 70.402649, accuracy: 0.925781
step: 750, loss: 165.765366, accuracy: 0.871094
step: 800, loss: 41.746349, accuracy: 0.941406
step: 850, loss: 39.642990, accuracy: 0.941406
step: 900, loss: 183.746918, accuracy: 0.855469
step: 950, loss: 234.977417, accuracy: 0.847656
step: 1000, loss: 40.009808, accuracy: 0.945312


<br/><h2> <b> Testing model accuracy using testing data.</b> </h2><br/>

In [17]:
# Test model on validation set.

pred = logistic_regression(inputs_test)

print("Test Accuracy: %f" % accuracy(pred, targets_test))

Test Accuracy: 0.913900
